In [1]:
import os
import numpy as np
import pandas as pd
import json
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv1D, MaxPooling1D, LSTM, Dense, Dropout,
    BatchNormalization, Concatenate
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, precision_recall_curve, roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.utils.class_weight import compute_class_weight
from scipy.signal import medfilt
from glob import glob
from collections import Counter

2026-02-20 19:27:29.942870: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/sonalimanoharan/Desktop/scientific_research/hw/lib/python3.12/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [2]:
base_path = "/Users/sonalimanoharan/Desktop/scientific_research/hw"
data_folders = ["data", "new_data"]
label_files = ["lables/labels.csv", "lables/lables_new.csv"]
save_model_path = os.path.join(base_path, "fair_model_swapped_labels")
os.makedirs(save_model_path, exist_ok=True)

In [3]:
SAMPLING_RATE = 50
WINDOW_DURATION = 3
WINDOW_SIZE = int(SAMPLING_RATE * WINDOW_DURATION)
STEP_SIZE = WINDOW_SIZE // 2
MIN_DURATION_SAMPLES = int(1.0 * SAMPLING_RATE)
FIXED_THRESHOLD = 0.5
VAL_FRACTION = 0.2

In [ ]:
def clean_humidity(df):
    if "humid" in df.columns:
        artifact_value = 79.1318359375
        df["humid"] = df["humid"].replace(artifact_value, np.nan)
        df["humid"] = df["humid"].interpolate(method='linear', limit_direction='both')
        df["humid"] = df["humid"].ffill().bfill()
    return df

In [ ]:
def load_recs():
    all_dfs = []
    for data_folder, label_file in zip(data_folders, label_files):
        data_path = os.path.join(base_path, data_folder, "*.csv")
        label_path = os.path.join(base_path, label_file)
        for fname in glob(data_path):
            df = pd.read_csv(fname)
            df = clean_humidity(df)
            subject_id = os.path.basename(fname).replace(".csv", "")
            all_dfs.append((fname, df, label_path, subject_id))
    return all_dfs

In [ ]:

def convert_to_binlabel(x):
    return 1 if x in ["Null", "dry"] else 0

def apply_labels(dfs):
    l_dfs = []
    for fname, df, label_path, subject_id in dfs:
        label_df = pd.read_csv(label_path)
        label_df["filename"] = label_df["datetime"].apply(lambda x: os.path.basename(str(x)).strip())
        file_basename = os.path.basename(fname).strip()
        matched_row = label_df[label_df["filename"].apply(lambda x: x.endswith(file_basename))]
        if matched_row.empty:
            continue
        df["label"] = "Null"
        label_info = json.loads(matched_row.iloc[0]["label"])
        for d in label_info:
            df.loc[d["start"]:d["end"], "label"] = d["timeserieslabels"][0]
        df["binlabel"] = df["label"].apply(convert_to_binlabel)
        df["subject"] = subject_id
        l_dfs.append(df)
    return l_dfs

In [ ]:
def convert_to_binlabel(x):
    return 1 if x in ["Null", "dry"] else 0

def apply_labels(dfs):
    l_dfs = []
    for fname, df, label_path, subject_id in dfs:
        label_df = pd.read_csv(label_path)
        label_df["filename"] = label_df["datetime"].apply(lambda x: os.path.basename(str(x)).strip())
        file_basename = os.path.basename(fname).strip()
        matched_row = label_df[label_df["filename"].apply(lambda x: x.endswith(file_basename))]
        if matched_row.empty:
            continue
        df["label"] = "Null"
        label_info = json.loads(matched_row.iloc[0]["label"])
        for d in label_info:
            df.loc[d["start"]:d["end"], "label"] = d["timeserieslabels"][0]
        df["binlabel"] = df["label"].apply(convert_to_binlabel)
        df["subject"] = subject_id
        l_dfs.append(df)
    return l_dfs

In [ ]:
def extract_magnitude_features(window):
    acc_mag = np.sqrt(window["acc_x"]**2 + window["acc_y"]**2 + window["acc_z"]**2)
    gyro_mag = np.sqrt(window["gyro_x"]**2 + window["gyro_y"]**2 + window["gyro_z"]**2)
    features = [np.mean(acc_mag), np.std(acc_mag), np.min(acc_mag), np.max(acc_mag),
                np.mean(gyro_mag), np.std(gyro_mag), np.min(gyro_mag), np.max(gyro_mag)]
    return np.column_stack([acc_mag, gyro_mag]), np.array(features)

In [ ]:
def extract_humidity_slope(window):
    if "humid" not in window.columns:
        return np.zeros(5)
    humid = window["humid"].values
    if len(humid) < 2:
        return np.zeros(5)
    slope = np.diff(humid)
    return np.array([np.mean(slope), np.std(slope), np.max(slope), np.min(slope), humid[-1] - humid[0]])

In [ ]:
def create_windows(df, window_size, step_size):
    magnitude_sequences, magnitude_features, humidity_features, labels = [], [], [], []
    for start in range(0, len(df) - window_size + 1, step_size):
        window = df.iloc[start:start + window_size]
        mag_seq, mag_feat = extract_magnitude_features(window)
        magnitude_sequences.append(mag_seq)
        magnitude_features.append(mag_feat)
        humidity_features.append(extract_humidity_slope(window))
        label_mode = window["binlabel"].mode()
        labels.append(label_mode.iloc[0] if not label_mode.empty else int(window["binlabel"].iloc[0]))
    return (np.array(magnitude_sequences), np.array(magnitude_features), np.array(humidity_features), np.array(labels))

In [ ]:

def focal_loss(alpha=0.25, gamma=2.0):
    def loss(y_true, y_pred):
        eps = 1e-7
        y_pred = tf.clip_by_value(y_pred, eps, 1.0 - eps)
        pt = tf.where(tf.equal(y_true, 1), y_pred, 1 - y_pred)
        return -tf.reduce_mean(alpha * tf.pow(1. - pt, gamma) * tf.math.log(pt))
    return loss



In [ ]:
def build_cnn_lstm_model(sequence_shape, feature_dim):
    seq_input = Input(shape=sequence_shape, name='magnitude_sequences')
    x = Conv1D(64, kernel_size=5, activation='relu', padding='same')(seq_input)
    x = BatchNormalization()(x)
    x = Conv1D(64, kernel_size=5, activation='relu', padding='same')(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Dropout(0.3)(x)
    x = Conv1D(128, kernel_size=3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Dropout(0.3)(x)
    x = LSTM(128, return_sequences=True)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = LSTM(64, return_sequences=False)(x)
    x = BatchNormalization()(x)
    cnn_lstm_output = Dropout(0.3)(x)
    feat_input = Input(shape=(feature_dim,), name='handcrafted_features')
    feat_dense = Dense(64, activation='relu')(feat_input)
    feat_dense = BatchNormalization()(feat_dense)
    feat_output = Dropout(0.3)(feat_dense)
    combined = Concatenate()([cnn_lstm_output, feat_output])
    combined = Dense(128, activation='relu')(combined)
    combined = BatchNormalization()(combined)
    combined = Dropout(0.4)(combined)
    combined = Dense(64, activation='relu')(combined)
    output = Dense(1, activation='sigmoid')(combined)
    return Model(inputs=[seq_input, feat_input], outputs=output)

In [ ]:

def apply_fixed_pipeline(y_pred_prob, threshold=0.5, kernel_size=5, min_duration_samples=None, step_size=None):
    y_pred_smooth = medfilt(y_pred_prob.flatten(), kernel_size=kernel_size)
    y_pred_binary = (y_pred_smooth > threshold).astype(int)
    if min_duration_samples is not None and step_size is not None:
        min_consecutive_windows = max(1, min_duration_samples // step_size)
        y_out = y_pred_binary.copy()
        i = 0
        while i < len(y_out):
            if y_out[i] == 1:
                start = i
                while i < len(y_out) and y_out[i] == 1:
                    i += 1
                if (i - start) < min_consecutive_windows:
                    y_out[start:i] = 0
            else:
                i += 1
        return y_out
    return y_pred_binary


In [ ]:

def threshold_from_validation(y_val, y_val_prob):
    precisions, recalls, thresholds = precision_recall_curve(y_val, y_val_prob)
    f1_scores = 2 * (precisions * recalls) / (precisions + recalls + 1e-10)
    best_idx = np.argmax(f1_scores)
    return thresholds[best_idx] if best_idx < len(thresholds) else 0.5

In [8]:
print("Loading data...")
all_dfs = load_recs()
labeled_dfs = apply_labels(all_dfs)
subjects = sorted(set(df["subject"].iloc[0] for df in labeled_dfs))
print(f"Found {len(subjects)} subjects")
results = []

Loading data...
Found 20 subjects


In [9]:
all_y_test_list = []
all_y_test_prob_list = []
for test_subject in subjects:
    print(f"\n{'='*60}")
    print(f"Test subject: {test_subject}")
    print(f"{'='*60}")

    test_dfs = [df for df in labeled_dfs if df["subject"].iloc[0] == test_subject]
    train_dfs = [df for df in labeled_dfs if df["subject"].iloc[0] != test_subject]

    X_seq_train, X_feat_train, y_train = [], [], []
    for df in train_dfs:
        mag_seq, mag_feat, hum_feat, labels = create_windows(df, WINDOW_SIZE, STEP_SIZE)
        X_seq_train.append(mag_seq)
        X_feat_train.append(np.hstack([mag_feat, hum_feat]))
        y_train.append(labels)
    X_seq_train = np.concatenate(X_seq_train, axis=0)
    X_feat_train = np.concatenate(X_feat_train, axis=0)
    y_train = np.concatenate(y_train, axis=0)

    X_seq_test, X_feat_test, y_test = [], [], []
    for df in test_dfs:
        mag_seq, mag_feat, hum_feat, labels = create_windows(df, WINDOW_SIZE, STEP_SIZE)
        X_seq_test.append(mag_seq)
        X_feat_test.append(np.hstack([mag_feat, hum_feat]))
        y_test.append(labels)
    X_seq_test = np.concatenate(X_seq_test, axis=0)
    X_feat_test = np.concatenate(X_feat_test, axis=0)
    y_test = np.concatenate(y_test, axis=0)

    n_train = len(y_train)
    val_size = int(n_train * VAL_FRACTION)
    idx = np.arange(n_train)
    np.random.seed(42)
    np.random.shuffle(idx)
    train_idx, val_idx = idx[val_size:], idx[:val_size]

    X_seq_tr, X_seq_val = X_seq_train[train_idx], X_seq_train[val_idx]
    X_feat_tr, X_feat_val = X_feat_train[train_idx], X_feat_train[val_idx]
    y_tr, y_val = y_train[train_idx], y_train[val_idx]

    scaler_seq = StandardScaler()
    scaler_feat = StandardScaler()
    X_seq_tr_flat = X_seq_tr.reshape(-1, X_seq_tr.shape[-1])
    scaler_seq.fit(X_seq_tr_flat)
    X_seq_tr = scaler_seq.transform(X_seq_tr_flat).reshape(X_seq_tr.shape)
    X_seq_val = scaler_seq.transform(X_seq_val.reshape(-1, X_seq_val.shape[-1])).reshape(X_seq_val.shape)
    X_seq_test_norm = scaler_seq.transform(X_seq_test.reshape(-1, X_seq_test.shape[-1])).reshape(X_seq_test.shape)
    X_feat_tr = scaler_feat.fit_transform(X_feat_tr)
    X_feat_val = scaler_feat.transform(X_feat_val)
    X_feat_test_norm = scaler_feat.transform(X_feat_test)

    class_weights = compute_class_weight("balanced", classes=np.unique(y_tr), y=y_tr)
    class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}
    alpha = 1 - np.mean(y_tr)

    sequence_shape = (X_seq_tr.shape[1], X_seq_tr.shape[2])
    feature_dim = X_feat_tr.shape[1]
    model = build_cnn_lstm_model(sequence_shape, feature_dim)
    model.compile(optimizer=Adam(learning_rate=0.001), loss=focal_loss(alpha=alpha, gamma=2.0), metrics=['accuracy'])

    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)
    model.fit(
        [X_seq_tr, X_feat_tr], y_tr,
        epochs=100, batch_size=32,
        validation_data=([X_seq_val, X_feat_val], y_val),
        class_weight=class_weight_dict, callbacks=[early_stopping], verbose=1
    )

    y_val_prob = model.predict([X_seq_val, X_feat_val], verbose=0)
    if hasattr(y_val_prob, 'numpy'):
        y_val_prob = y_val_prob.numpy().flatten()
    else:
        y_val_prob = np.array(y_val_prob).flatten()
    optimal_threshold = threshold_from_validation(y_val, y_val_prob)

    y_test_prob = model.predict([X_seq_test_norm, X_feat_test_norm], verbose=0)
    if hasattr(y_test_prob, 'numpy'):
        y_test_prob = y_test_prob.numpy().flatten()
    else:
        y_test_prob = np.array(y_test_prob).flatten()
    all_y_test_list.append(y_test)
    all_y_test_prob_list.append(y_test_prob)

    y_pred_fixed = apply_fixed_pipeline(y_test_prob, threshold=FIXED_THRESHOLD, min_duration_samples=MIN_DURATION_SAMPLES, step_size=STEP_SIZE)
    y_pred_val_threshold = apply_fixed_pipeline(y_test_prob, threshold=optimal_threshold, min_duration_samples=MIN_DURATION_SAMPLES, step_size=STEP_SIZE)

    fair_f1_fixed = f1_score(y_test, y_pred_fixed)
    fair_acc_fixed = accuracy_score(y_test, y_pred_fixed)
    fair_f1_val = f1_score(y_test, y_pred_val_threshold)
    fair_acc_val = accuracy_score(y_test, y_pred_val_threshold)

    print(f"  Fair F1 (fixed 0.5 + median filter): {fair_f1_fixed:.4f}  Acc: {fair_acc_fixed:.4f}")
    print(f"  Fair F1 (val-chosen threshold={optimal_threshold:.3f}): {fair_f1_val:.4f}  Acc: {fair_acc_val:.4f}")

    results.append({
        "subject": test_subject,
        "fair_f1_fixed": fair_f1_fixed,
        "fair_acc_fixed": fair_acc_fixed,
        "fair_f1_val_threshold": fair_f1_val,
        "fair_acc_val_threshold": fair_acc_val,
        "val_threshold": optimal_threshold
    })

    model.save(os.path.join(save_model_path, f"compare_swapped_{test_subject}.h5"))


Test subject: 2024-12-04-18-49-30_c5c72868-633a-4672-8bdd-3a457f994ddb
Epoch 1/100
1163/1163 ━━━━━━━━━━━━━━━━━━━━ 100s 81ms/step - accuracy: 0.9435 - loss: 0.0020 - val_accuracy: 0.9614 - val_loss: 0.0016
Epoch 2/100
1163/1163 ━━━━━━━━━━━━━━━━━━━━ 86s 74ms/step - accuracy: 0.9601 - loss: 0.0015 - val_accuracy: 0.9658 - val_loss: 0.0011
Epoch 3/100
1163/1163 ━━━━━━━━━━━━━━━━━━━━ 98s 85ms/step - accuracy: 0.9625 - loss: 0.0013 - val_accuracy: 0.8983 - val_loss: 0.0019
Epoch 4/100
1163/1163 ━━━━━━━━━━━━━━━━━━━━ 95s 82ms/step - accuracy: 0.9622 - loss: 0.0013 - val_accuracy: 0.9675 - val_loss: 9.6284e-04
Epoch 5/100
1163/1163 ━━━━━━━━━━━━━━━━━━━━ 97s 84ms/step - accuracy: 0.9636 - loss: 0.0013 - val_accuracy: 0.9696 - val_loss: 9.1139e-04
Epoch 6/100
1163/1163 ━━━━━━━━━━━━━━━━━━━━ 101s 87ms/step - accuracy: 0.9658 - loss: 0.0012 - val_accuracy: 0.9690 - val_loss: 9.2747e-04
Epoch 7/100
1163/1163 ━━━━━━━━━━━━━━━━━━━━ 101s 87ms/step - accuracy: 0.9651 - loss: 0.0012 - val_accuracy: 0.9625 -

  Fair F1 (fixed 0.5 + median filter): 0.9580  Acc: 0.9198
  Fair F1 (val-chosen threshold=0.493): 0.9575  Acc: 0.9188

Test subject: 2024-12-08-21-41-18_c1291a19-92af-431e-9608-6044389d26b0
Epoch 1/100
1156/1156 ━━━━━━━━━━━━━━━━━━━━ 99s 81ms/step - accuracy: 0.9427 - loss: 0.0021 - val_accuracy: 0.9640 - val_loss: 0.0010
Epoch 2/100
1156/1156 ━━━━━━━━━━━━━━━━━━━━ 97s 84ms/step - accuracy: 0.9588 - loss: 0.0014 - val_accuracy: 0.9653 - val_loss: 9.8223e-04
Epoch 3/100
1156/1156 ━━━━━━━━━━━━━━━━━━━━ 97s 83ms/step - accuracy: 0.9649 - loss: 0.0013 - val_accuracy: 0.9652 - val_loss: 9.7009e-04
Epoch 4/100
1156/1156 ━━━━━━━━━━━━━━━━━━━━ 91s 78ms/step - accuracy: 0.9665 - loss: 0.0012 - val_accuracy: 0.9661 - val_loss: 9.8510e-04
Epoch 5/100
1156/1156 ━━━━━━━━━━━━━━━━━━━━ 89s 77ms/step - accuracy: 0.9667 - loss: 0.0012 - val_accuracy: 0.9682 - val_loss: 9.0766e-04
Epoch 6/100
1156/1156 ━━━━━━━━━━━━━━━━━━━━ 89s 77ms/step - accuracy: 0.9670 - loss: 0.0012 - val_accuracy: 0.9685 - val_loss: 8.

  Fair F1 (fixed 0.5 + median filter): 0.9472  Acc: 0.8997
  Fair F1 (val-chosen threshold=0.488): 0.9472  Acc: 0.8997

Test subject: 2024-12-10-19-42-27_4734a243-b638-4004-aa82-c698f3ef7aba
Epoch 1/100
1150/1150 ━━━━━━━━━━━━━━━━━━━━ 95s 78ms/step - accuracy: 0.9470 - loss: 0.0020 - val_accuracy: 0.9643 - val_loss: 0.0012
Epoch 2/100
1150/1150 ━━━━━━━━━━━━━━━━━━━━ 89s 78ms/step - accuracy: 0.9610 - loss: 0.0014 - val_accuracy: 0.9661 - val_loss: 0.0011
Epoch 3/100
1150/1150 ━━━━━━━━━━━━━━━━━━━━ 90s 78ms/step - accuracy: 0.9630 - loss: 0.0013 - val_accuracy: 0.9641 - val_loss: 0.0011
Epoch 4/100
1150/1150 ━━━━━━━━━━━━━━━━━━━━ 89s 77ms/step - accuracy: 0.9655 - loss: 0.0012 - val_accuracy: 0.9623 - val_loss: 0.0011
Epoch 5/100
1150/1150 ━━━━━━━━━━━━━━━━━━━━ 89s 77ms/step - accuracy: 0.9648 - loss: 0.0012 - val_accuracy: 0.9669 - val_loss: 9.0656e-04
Epoch 6/100
1150/1150 ━━━━━━━━━━━━━━━━━━━━ 90s 78ms/step - accuracy: 0.9662 - loss: 0.0012 - val_accuracy: 0.9653 - val_loss: 0.0011
Epoch 7

  Fair F1 (fixed 0.5 + median filter): 0.9633  Acc: 0.9292
  Fair F1 (val-chosen threshold=0.505): 0.9639  Acc: 0.9304

Test subject: 2025-01-18-13-08-43_449ee30d-3245-47ca-9769-752cf0d2edb7
Epoch 1/100
1153/1153 ━━━━━━━━━━━━━━━━━━━━ 114s 94ms/step - accuracy: 0.9400 - loss: 0.0022 - val_accuracy: 0.9647 - val_loss: 0.0012
Epoch 2/100
1153/1153 ━━━━━━━━━━━━━━━━━━━━ 84s 73ms/step - accuracy: 0.9590 - loss: 0.0016 - val_accuracy: 0.9622 - val_loss: 0.0012
Epoch 3/100
1153/1153 ━━━━━━━━━━━━━━━━━━━━ 85s 74ms/step - accuracy: 0.9615 - loss: 0.0014 - val_accuracy: 0.9641 - val_loss: 0.0011
Epoch 4/100
1153/1153 ━━━━━━━━━━━━━━━━━━━━ 85s 74ms/step - accuracy: 0.9628 - loss: 0.0013 - val_accuracy: 0.9653 - val_loss: 0.0013
Epoch 5/100
1153/1153 ━━━━━━━━━━━━━━━━━━━━ 92s 80ms/step - accuracy: 0.9617 - loss: 0.0014 - val_accuracy: 0.9666 - val_loss: 9.6948e-04
Epoch 6/100
1153/1153 ━━━━━━━━━━━━━━━━━━━━ 85s 74ms/step - accuracy: 0.9633 - loss: 0.0013 - val_accuracy: 0.9661 - val_loss: 0.0010
Epoch 

  Fair F1 (fixed 0.5 + median filter): 0.9786  Acc: 0.9586
  Fair F1 (val-chosen threshold=0.544): 0.9792  Acc: 0.9599

Test subject: 2025-01-18-22-38-29_37959204-490b-4cd9-b647-94e743071951
Epoch 1/100
1152/1152 ━━━━━━━━━━━━━━━━━━━━ 89s 72ms/step - accuracy: 0.9367 - loss: 0.0025 - val_accuracy: 0.9652 - val_loss: 0.0012
Epoch 2/100
1152/1152 ━━━━━━━━━━━━━━━━━━━━ 85s 74ms/step - accuracy: 0.9547 - loss: 0.0017 - val_accuracy: 0.9671 - val_loss: 0.0011
Epoch 3/100
1152/1152 ━━━━━━━━━━━━━━━━━━━━ 86s 75ms/step - accuracy: 0.9582 - loss: 0.0015 - val_accuracy: 0.9674 - val_loss: 0.0011
Epoch 4/100
1152/1152 ━━━━━━━━━━━━━━━━━━━━ 86s 74ms/step - accuracy: 0.9615 - loss: 0.0014 - val_accuracy: 0.9649 - val_loss: 0.0011
Epoch 5/100
1152/1152 ━━━━━━━━━━━━━━━━━━━━ 86s 75ms/step - accuracy: 0.9628 - loss: 0.0014 - val_accuracy: 0.9674 - val_loss: 0.0010
Epoch 6/100
1152/1152 ━━━━━━━━━━━━━━━━━━━━ 87s 75ms/step - accuracy: 0.9632 - loss: 0.0014 - val_accuracy: 0.9658 - val_loss: 0.0012
Epoch 7/100

...

  Fair F1 (fixed 0.5 + median filter): 0.9939  Acc: 0.9881
  Fair F1 (val-chosen threshold=0.528): 0.9948  Acc: 0.9899

Test subject: ab0a6b0c-b0f2-4bda-8806-a4e39175f027
Epoch 1/100
1152/1152 ━━━━━━━━━━━━━━━━━━━━ 101s 82ms/step - accuracy: 0.9480 - loss: 0.0020 - val_accuracy: 0.9590 - val_loss: 0.0013
Epoch 2/100
1152/1152 ━━━━━━━━━━━━━━━━━━━━ 92s 80ms/step - accuracy: 0.9593 - loss: 0.0016 - val_accuracy: 0.9603 - val_loss: 0.0013
Epoch 3/100
1152/1152 ━━━━━━━━━━━━━━━━━━━━ 93s 81ms/step - accuracy: 0.9600 - loss: 0.0015 - val_accuracy: 0.9540 - val_loss: 0.0013
Epoch 4/100
1152/1152 ━━━━━━━━━━━━━━━━━━━━ 93s 80ms/step - accuracy: 0.9609 - loss: 0.0014 - val_accuracy: 0.9651 - val_loss: 0.0011
Epoch 5/100
1152/1152 ━━━━━━━━━━━━━━━━━━━━ 93s 81ms/step - accuracy: 0.9631 - loss: 0.0014 - val_accuracy: 0.9643 - val_loss: 0.0011
Epoch 6/100
1152/1152 ━━━━━━━━━━━━━━━━━━━━ 95s 82ms/step - accuracy: 0.9626 - loss: 0.0014 - val_accuracy: 0.9624 - val_loss: 0.0011
Epoch 7/100
1152/1152 ━━━━━━━━

  Fair F1 (fixed 0.5 + median filter): 0.9919  Acc: 0.9841
  Fair F1 (val-chosen threshold=0.481): 0.9913  Acc: 0.9829

Test subject: ebc39e6c-2770-4821-a747-c174a7855b30
Epoch 1/100
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 157s 131ms/step - accuracy: 0.9444 - loss: 0.0021 - val_accuracy: 0.9633 - val_loss: 0.0013
Epoch 2/100
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 146s 127ms/step - accuracy: 0.9579 - loss: 0.0016 - val_accuracy: 0.9639 - val_loss: 0.0012
Epoch 3/100
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 147s 127ms/step - accuracy: 0.9595 - loss: 0.0015 - val_accuracy: 0.9643 - val_loss: 0.0012
Epoch 4/100
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 149s 129ms/step - accuracy: 0.9619 - loss: 0.0015 - val_accuracy: 0.9649 - val_loss: 0.0012
Epoch 5/100
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 149s 129ms/step - accuracy: 0.9615 - loss: 0.0014 - val_accuracy: 0.9651 - val_loss: 0.0011
Epoch 6/100
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 150s 130ms/step - accuracy: 0.9625 - loss: 0.0014 - val_accuracy: 0.9649 - val_loss: 0.0012
Epoch 7/100
1151/11

  Fair F1 (fixed 0.5 + median filter): 0.9904  Acc: 0.9814
  Fair F1 (val-chosen threshold=0.502): 0.9904  Acc: 0.9814

Test subject: ebff48bd-b1c8-44e3-af35-0941b6c405b1
Epoch 1/100
1150/1150 ━━━━━━━━━━━━━━━━━━━━ 140s 116ms/step - accuracy: 0.9480 - loss: 0.0021 - val_accuracy: 0.9589 - val_loss: 0.0014
Epoch 2/100
1150/1150 ━━━━━━━━━━━━━━━━━━━━ 126s 110ms/step - accuracy: 0.9591 - loss: 0.0016 - val_accuracy: 0.9655 - val_loss: 0.0011
Epoch 3/100
1150/1150 ━━━━━━━━━━━━━━━━━━━━ 125s 108ms/step - accuracy: 0.9605 - loss: 0.0015 - val_accuracy: 0.9578 - val_loss: 0.0015
Epoch 4/100
1150/1150 ━━━━━━━━━━━━━━━━━━━━ 122s 106ms/step - accuracy: 0.9606 - loss: 0.0015 - val_accuracy: 0.9664 - val_loss: 0.0010
Epoch 5/100
1150/1150 ━━━━━━━━━━━━━━━━━━━━ 120s 104ms/step - accuracy: 0.9619 - loss: 0.0014 - val_accuracy: 0.9662 - val_loss: 0.0011
Epoch 6/100
1150/1150 ━━━━━━━━━━━━━━━━━━━━ 119s 103ms/step - accuracy: 0.9629 - loss: 0.0014 - val_accuracy: 0.9660 - val_loss: 0.0011
Epoch 7/100
1150/11

  Fair F1 (fixed 0.5 + median filter): 0.9856  Acc: 0.9719
  Fair F1 (val-chosen threshold=0.493): 0.9860  Acc: 0.9727

Test subject: fa94190b-92d3-484c-8133-744b797dfc81
Epoch 1/100
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 110s 90ms/step - accuracy: 0.9439 - loss: 0.0022 - val_accuracy: 0.9615 - val_loss: 0.0013
Epoch 2/100
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 102s 89ms/step - accuracy: 0.9570 - loss: 0.0016 - val_accuracy: 0.9624 - val_loss: 0.0013
Epoch 3/100
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 102s 88ms/step - accuracy: 0.9595 - loss: 0.0015 - val_accuracy: 0.9662 - val_loss: 0.0010
Epoch 4/100
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 102s 89ms/step - accuracy: 0.9607 - loss: 0.0014 - val_accuracy: 0.9679 - val_loss: 9.9194e-04
Epoch 5/100
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 102s 89ms/step - accuracy: 0.9600 - loss: 0.0015 - val_accuracy: 0.9665 - val_loss: 0.0011
Epoch 6/100
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 102s 89ms/step - accuracy: 0.9615 - loss: 0.0014 - val_accuracy: 0.9615 - val_loss: 0.0012
Epoch 7/100
1151/1151

  Fair F1 (fixed 0.5 + median filter): 0.9893  Acc: 0.9792
  Fair F1 (val-chosen threshold=0.495): 0.9909  Acc: 0.9824


In [10]:
df_results = pd.DataFrame(results)
print("\n" + "="*60)
print("FAIR F1 SUMMARY — SWAPPED LABELS (F1 = no-handwash detection)")
print("="*60)
print(df_results.to_string())
print("\n--- Means ---")
print(f"Mean Fair F1 [no-handwash] (fixed):     {df_results['fair_f1_fixed'].mean():.4f}")
print(f"Mean Fair F1 [no-handwash] (val threshold):   {df_results['fair_f1_val_threshold'].mean():.4f}")
print(f"Mean Fair Accuracy (fixed):                   {df_results['fair_acc_fixed'].mean():.4f}")
print(f"Mean Fair Accuracy (val threshold):           {df_results['fair_acc_val_threshold'].mean():.4f}")
df_results.to_csv(os.path.join(save_model_path, "compare_swapped_labels_results.csv"), index=False)
print(f"\nResults saved to {os.path.join(save_model_path, 'compare_swapped_labels_results.csv')}")


FAIR F1 SUMMARY — SWAPPED LABELS (F1 = no-handwash detection)
                                                     subject  fair_f1_fixed  fair_acc_fixed  fair_f1_val_threshold  fair_acc_val_threshold  val_threshold
0   2024-12-04-18-49-30_c5c72868-633a-4672-8bdd-3a457f994ddb       0.957974        0.919781               0.957475                0.918784       0.492592
1   2024-12-08-21-41-18_c1291a19-92af-431e-9608-6044389d26b0       0.947223        0.899738               0.947223                0.899738       0.487971
2   2024-12-10-19-42-27_4734a243-b638-4004-aa82-c698f3ef7aba       0.963312        0.929249               0.963905                0.930435       0.504665
3   2025-01-18-13-08-43_449ee30d-3245-47ca-9769-752cf0d2edb7       0.978622        0.958612               0.979212                0.959866       0.544296
4   2025-01-18-22-38-29_37959204-490b-4cd9-b647-94e743071951       0.987331        0.975092               0.970931                0.944059       0.545768
5   2025-01-1

In [ ]:
y_all = np.concatenate(all_y_test_list)
prob_all = np.concatenate(all_y_test_prob_list)
fpr, tpr, thresholds = roc_curve(y_all, prob_all)
roc_auc = auc(fpr, tpr)
J = tpr - fpr
optimal_idx = np.argmax(J)
optimal_threshold_youden = thresholds[optimal_idx]

fig, axes = plt.subplots(1, 2, figsize=(12, 5))
axes[0].plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC (AUC = {roc_auc:.3f})')
axes[0].plot([0, 1], [0, 1], color='navy', lw=1, linestyle='--')
axes[0].scatter(fpr[optimal_idx], tpr[optimal_idx], color='red', s=80, zorder=5,
                label=f"Optimal threshold = {optimal_threshold_youden:.3f}")
axes[0].set_xlabel('False Positive Rate')
axes[0].set_ylabel('True Positive Rate')
axes[0].set_title('ROC Curve (pooled test across LOSO) — Swapped labels')
axes[0].legend(loc='lower right')
axes[0].grid(True, alpha=0.3)
axes[1].bar(range(len(df_results)), df_results['val_threshold'], color='steelblue', alpha=0.8)
axes[1].axhline(y=optimal_threshold_youden, color='red', linestyle='--', label=f'Youden optimal ({optimal_threshold_youden:.3f})')
axes[1].set_xlabel('Subject index')
axes[1].set_ylabel('Threshold')
axes[1].set_title('Validation-chosen threshold per subject')
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()